In [1]:
import pandas as pd
from datetime import datetime, timedelta #datetime opera fechas con +,-, mayor qué, menor qué. Timedelta le ayuda a sumar o restar días a una fecha
import os
import re
import matplotlib.pyplot as plt #librería para graficar 

In [5]:
def generar_lista_fechas(fecha_inicio, fecha_fin):
    

    lista_archivos = []  # Crea lista vacía para almacenar los nombres de los archivos
    aux = fecha_inicio  # Crea variable auxiliar para recorrer las fechas
    
    while aux <= fecha_fin:
        file = aux.strftime("%d_%m_%Y") + "ProductosMasVendidos.xls"
        lista_archivos.append(file)  # Coloca file al final de la lista
        aux += timedelta(days=1)  # Recorre la fecha un día hacia adelante

    # Revisamos si la lista está vacía
    if len(lista_archivos) == 0:
        print("No se generaron nombres de archivos")
    else:
        #print(f"Se generaron los siguientes {len(lista_archivos)} nombres de archivos:")
        for archivo in lista_archivos:
            #print(archivo)
            pass

    return lista_archivos

# Definir las fechas de inicio y fin
fecha_inicio = datetime(2024, 1, 1)
fecha_fin = datetime(2024, 12, 31)
print("Fechas de inicio y fin definidas.")
print(f"Fecha de inicio: {fecha_inicio}")
print(f"Fecha de fin: {fecha_fin}")
# Llamar a la función y obtener la lista de archivos
lista_archivos = generar_lista_fechas(fecha_inicio, fecha_fin)

Fechas de inicio y fin definidas.
Fecha de inicio: 2024-01-01 00:00:00
Fecha de fin: 2024-12-31 00:00:00


In [6]:
def obtener_archivos(ruta):
    archivos_anuales = [] # Lista donde se almacenarán los archivos encontrados
    
    # Expresión regular para validar que los archivos tengan el formato correcto
    patron = re.compile(r"^\d{2}_\d{2}_\d{4}ProductosMasVendidos\.xls$")  # Patrón para validar el nombre de archivo

    # Recorre todas las carpetas dentro de la ruta base (cada carpeta representa un mes)
    for mes in os.listdir(ruta):
        ruta_mes = os.path.join(ruta, mes) # Construye la ruta completa de la carpeta del mes

        # Verifica si es un directorio antes de procesarlo
        if os.path.isdir(ruta_mes):
            
            # Filtra los archivos de la carpeta del mes que cumplen con el patrón definido
            archivos_mes = [f for f in os.listdir(ruta_mes) if patron.match(f)]
            archivos_anuales.extend(archivos_mes)  # Agrega los archivos encontrados a la lista total 

    #print(f"Se encontraron {len(archivos_anuales)} archivos en total.")
    return archivos_anuales

# ruta donde están las carpetas de cada mes
ruta = "C:/Users/david/Documents/Desarrollo/Datos2"

# Llamar a la función
archivos_totales = obtener_archivos(ruta)

In [7]:
def comparar_listas_existencia(lista_fechas, archivos_totales):
    
    return set(lista_fechas).issubset(archivos_totales)  # Eliminamos conversión redundante

# Generar la lista de archivos esperados
lista_fechas = generar_lista_fechas(datetime(2024, 1, 1), datetime(2024, 12, 31))

# Obtener los archivos encontrados en la carpeta
archivos_totales = obtener_archivos("C:/Users/david/Documents/Desarrollo/Datos2")

if not archivos_totales:
    print("No se encontraron archivos en la ruta. Verifica que la carpeta no esté vacía o que los nombres sean correctos.")
elif comparar_listas_existencia(lista_fechas, archivos_totales):
    print("Todos los archivos generados están presentes en la lista de fechas.")
else:
    print("No todos los archivos generados están presentes en la lista de fechas.")


Todos los archivos generados están presentes en la lista de fechas.


In [8]:
def recoleccion_archivos(ruta):
    dataframes = []  # Lista para almacenar los DataFrames de cada archivo

    # Recorrer cada carpeta de los meses dentro de la ruta base
    for mes in os.listdir(ruta):
        ruta_mes = os.path.join(ruta, mes)
        print(mes)

        if os.path.isdir(ruta_mes):  # Verificar que sea una carpeta
            archivos = [f for f in os.listdir(ruta_mes) if f.endswith('.xls')]

            for archivo in archivos:
                # Intentamos procesar el archivo Excel
                try:
                    #print(f"Procesando el archivo: {archivo}")
                    # Leer el archivo de Excel, omitiendo las 3 primeras filas y seleccionando columnas específicas
                    df = pd.read_excel(os.path.join(ruta_mes, archivo), skiprows=3, usecols=[4, 5, 6, 7])

                    # Extraer la fecha del nombre del archivo usando una expresión regular
                    match = re.search(r'(\d{1,2}_\d{1,2}_\d{4})', archivo)

                    if match:
                        fecha_str = match.group(1)
                        fecha_reporte = datetime.strptime(fecha_str, '%d_%m_%Y').date()

                        if df.empty:
                            print(f"El archivo {archivo} está vacío.")
                        else:
                            # Agregar la fecha del reporte como una columna adicional en el DataFrame
                            df['Fecha'] = fecha_reporte

                            # Reordenar las columnas para que la fecha esté al final
                            columnas = list(df.columns)
                            columnas.append(columnas.pop(columnas.index('Fecha')))
                            df = df[columnas]

                            # Agregar el DataFrame a la lista
                            dataframes.append(df)
                    else:
                        print(f"No se encontró una fecha válida en el nombre del archivo '{archivo}'")
                except Exception as e:
                    print(f"Error al procesar el archivo {archivo}: {e}")


# Definir la ruta base que contiene las 12 carpetas de los meses
ruta = "C:/Users/david/Documents/Desarrollo/Datos2"

# Llamar a la función para procesar todos los archivos del año
df_anual = recoleccion_archivos(ruta)


Abril24
Agosto24
Diciembre24
El archivo 25_12_2024ProductosMasVendidos.xls está vacío.
Enero24
El archivo 01_01_2024ProductosMasVendidos.xls está vacío.
Febrero24
Julio24
Junio24
Marzo24
Mayo24
El archivo 01_05_2024ProductosMasVendidos.xls está vacío.
Noviembre24
Octubre24
reporte_anual.csv
Septiembre24


In [9]:
#obtener una lista de los nombres de las carpetas
def listar_meses(ruta):
    # Se retorna una lista de los nombres de las carpetas (meses) que existen en la ruta indicada
    return [mes for mes in os.listdir(ruta) if os.path.isdir(os.path.join(ruta, mes))]

In [10]:
#obtener una lista de todos los archivos que terminan con la extensión .xls
def listar_archivos_xls(ruta_mes):
    # Se retorna una lista de los archivos con extensión '.xls' dentro del directorio especificado
    return [f for f in os.listdir(ruta_mes) if f.endswith('.xls')]

In [11]:
#extraer la fecha de un nombre de archivo, asumiendo que la fecha está en formato dd_mm_yyyy
def extraer_fecha_desde_nombre(archivo):  # Buscar una fecha con formato 'DD_MM_YYYY' dentro del nombre del archivo
    match = re.search(r'(\d{1,2}_\d{1,2}_\d{4})', archivo)
    if match:
        # Convertir la fecha extraída de texto a un objeto de fecha
        return datetime.strptime(match.group(1), '%d_%m_%Y').date()
    return None # Retornar None si no se encuentra una fecha válida

In [12]:
#leer un archivo Excel, realizar algunas transformaciones en los datos y devolver un DataFrame con la información procesada
def procesar_archivo_excel(ruta_archivo, fecha_reporte):
    try:
        # Cargar el archivo Excel, omitiendo las primeras 3 filas y seleccionando columnas específicas
        df = pd.read_excel(ruta_archivo, skiprows=3, usecols=[4, 5, 6, 7])
        # Verificar si el DataFrame está vacío
        if df.empty:
            print(f"El archivo {os.path.basename(ruta_archivo)} está vacío.")
            return None
        
        # Agregar la fecha de reporte como una nueva columna
        df['Fecha'] = fecha_reporte
        
        # Reorganizar las columnas para que la fecha sea la última
        columnas = list(df.columns) # Obtener la lista de nombres de columnas
        columnas.append(columnas.pop(columnas.index('Fecha'))) # Mover la columna 'Fecha' al final
        
        return df[columnas]# Retornar el DataFrame reorganizado
    
    except Exception as e:
        # Capturar y mostrar cualquier error que ocurra durante el procesamiento
        print(f"Error al procesar el archivo {os.path.basename(ruta_archivo)}: {e}")
        return None


In [14]:
#reunir y procesar todos los archivos Excel (.xls) que se encuentran dentro de una estructura de carpetas organizadas por meses.
def recoleccion_archivos(ruta):


    dataframes = []  # Lista donde se almacenarán los DataFrames procesados
    
    # Iteramos sobre las carpetas de los meses dentro de la ruta base
    for mes in listar_meses(ruta):
        ruta_mes = os.path.join(ruta, mes)  # Construimos la ruta de la carpeta del mes
        print(f"Procesando mes: {mes}")  # Mensaje informativo
        
        # Iteramos sobre los archivos Excel dentro de la carpeta del mes
        for archivo in listar_archivos_xls(ruta_mes):
            #print(f"Procesando archivo: {archivo}")  # Mensaje informativo
            fecha_reporte = extraer_fecha_desde_nombre(archivo)  # Extraemos la fecha desde el nombre del archivo
            
            if fecha_reporte:  # Verificamos si la fecha fue extraída correctamente
                # Procesamos el archivo Excel y obtenemos un DataFrame
                df = procesar_archivo_excel(os.path.join(ruta_mes, archivo), fecha_reporte)
                
                if df is not None:  # Verificamos si el DataFrame se generó correctamente
                    dataframes.append(df)  # Agregamos el DataFrame a la lista
            else:
                print(f"No se encontró una fecha válida en el nombre del archivo '{archivo}'")  # Mensaje de advertencia
    
    return dataframes  # Retornamos la lista de DataFrames procesados

# Definir la ruta base
ruta = "C:/Users/david/Documents/Desarrollo/Datos2"

# Llamar a la función principal para recolectar y procesar los archivos Excel
df_anual = recoleccion_archivos(ruta)



Procesando mes: Abril24
Procesando mes: Agosto24
Procesando mes: Diciembre24
El archivo 25_12_2024ProductosMasVendidos.xls está vacío.
Procesando mes: Enero24
El archivo 01_01_2024ProductosMasVendidos.xls está vacío.
Procesando mes: Febrero24
Procesando mes: Julio24
Procesando mes: Junio24
Procesando mes: Marzo24
Procesando mes: Mayo24
El archivo 01_05_2024ProductosMasVendidos.xls está vacío.
Procesando mes: Noviembre24
Procesando mes: Octubre24
Procesando mes: Septiembre24
